# Text Extraction from Dutch News Articles

This notebook extracts clean, readable text from the NOS Dutch news articles feather dataset. The main focus is on properly cleaning HTML content and storing articles in a single SQLite database with native columns for fast querying.

## Key Features

1. Improved HTML Cleaning: Proper spacing between HTML elements to avoid word concatenation
2. SQLite Storage: One compact database file with a row per article
3. Database-Native Schema: Explicit columns for core fields
4. Batch Processing: Efficient processing with transaction-per-batch
5. Indexes: Fast filtering and sorting by publication time

## Process Overview

1. Load the feather dataset and examine structure
2. Implement improved HTML cleaning with proper text separation
3. Create text preprocessing and validation functions
4. Store cleaned articles directly in SQLite with native columns
5. Process the full dataset in batches

## Import Required Libraries

Import all necessary libraries for dataset processing, HTML cleaning, file operations, and text processing.

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
from pathlib import Path
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import sqlite3
from contextlib import closing

# Display settings for better data exploration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)

print("✅ Libraries imported successfully!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🧹 BeautifulSoup available for HTML cleaning")
print(f"📁 Path and file operations ready")
print(f"⏰ DateTime handling configured")
print(f"🗄️ SQLite ready")

✅ Libraries imported successfully!
📊 Pandas version: 2.3.1
🧹 BeautifulSoup available for HTML cleaning
📁 Path and file operations ready
⏰ DateTime handling configured
🗄️ SQLite ready


## Load the Feather Dataset

Load the NOS Dutch news articles dataset and examine its structure, fields, and content types.

In [2]:
# Load the feather dataset
file_path = "data/NOS_NL_articles_2015_mar_2025.feather"

print(f"📂 Loading dataset from: {file_path}")
print(f"📁 File exists: {os.path.exists(file_path)}")

if os.path.exists(file_path):
    # Get file size
    file_size = os.path.getsize(file_path)
    print(f"💾 File size: {file_size / (1024**2):.2f} MB")
    
    # Load the dataset
    print("🔄 Loading data...")
    df = pd.read_feather(file_path)
    
    print(f"\n✅ Dataset loaded successfully!")
    print(f"📊 Shape: {df.shape} (rows, columns)")
    print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # Display basic information
    print(f"\n📋 Dataset Info:")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Date range: {df['published_time'].min()} to {df['published_time'].max()}")
    
    # Sample data
    print(f"\n🔍 Sample data:")
    print(df[['title', 'description', 'published_time']].head(3))
    
    # Check content field specifically
    if 'content' in df.columns:
        print(f"\n🌐 HTML Content Analysis:")
        content_with_html = df['content'].notna().sum()
        print(f"   Articles with content: {content_with_html:,}")
        
        # Show sample HTML content
        sample_content = df[df['content'].notna()]['content'].iloc[0][:500]
        print(f"   Sample content (first 500 chars): {sample_content}...")
    
else:
    print("❌ File not found! Please check the file path.")
    df = None

📂 Loading dataset from: data/NOS_NL_articles_2015_mar_2025.feather
📁 File exists: True
💾 File size: 503.98 MB
🔄 Loading data...

✅ Dataset loaded successfully!
📊 Shape: (295259, 11) (rows, columns)
💾 Memory usage: 1361.08 MB

📋 Dataset Info:
   Columns: ['channel', 'url', 'type', 'title', 'keywords', 'section', 'description', 'published_time', 'modified_time', 'image', 'content']
   Date range: 2015-01-01 00:32:52 to 2025-03-31 23:45:01

🔍 Sample data:
                                                         title  \
1948                                   Euro nu ook in Litouwen   
1949  Start 2015 vol vreugde maar ook met gewonden en inzet ME   
1950            Letland nieuwe voorzitter van de Europese Unie   

                                                                                              description  \
1948  Vanaf vandaag betalen ze in Litouwen met de euro. Alle Baltische landen hebben nu de Europese munt.   
1949  Nederland is met oliebollen en vuurwerk het nieuwe jaa

## HTML Content Cleaning with Proper Text Separation

Create improved HTML cleaning functions that prevent word concatenation by ensuring proper spacing between HTML elements, especially headings and paragraphs.

In [3]:
def clean_html_content_improved(html_content):
    """
    Clean HTML content with proper text separation to prevent word concatenation.
    
    This addresses issues like <h2>Bloedneus</h2><h2>Adele</h2><p>Kirsten Sokol:</p>
    becoming "bloadneusadelekirsten" by ensuring proper spacing between elements.
    
    Args:
        html_content (str): Raw HTML content from articles
        
    Returns:
        str: Clean text with proper word separation
    """
    if pd.isna(html_content) or not html_content:
        return ""
    
    try:
        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Remove script and style elements completely
        for script in soup(["script", "style", "noscript"]):
            script.decompose()
        
        # Add spacing around block-level elements to prevent concatenation
        block_elements = [
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6',  # Headings
            'p', 'div', 'article', 'section',     # Paragraphs and containers
            'blockquote', 'pre', 'address',       # Text blocks
            'li', 'dd', 'dt',                     # List items
            'tr', 'td', 'th',                     # Table elements
            'header', 'footer', 'nav', 'aside'    # Semantic elements
        ]
        
        # Add separators around block elements
        for tag_name in block_elements:
            for tag in soup.find_all(tag_name):
                # Add space before the tag content
                if tag.string:
                    tag.string = f" {tag.string.strip()} "
                elif tag.get_text():
                    # For tags with mixed content, wrap in spaces
                    tag.insert_before(" ")
                    tag.insert_after(" ")
        
        # Get text content with separator
        text = soup.get_text(separator=' ')
        
        # Clean up whitespace but preserve sentence structure
        lines = []
        for line in text.splitlines():
            line = line.strip()
            if line:
                # Replace multiple spaces with single space
                line = re.sub(r'\s+', ' ', line)
                lines.append(line)
        
        # Join lines with space and clean up
        text = ' '.join(lines)
        
        # Final cleanup - remove excessive spacing
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    
    except Exception as e:
        print(f"Error cleaning HTML: {e}")
        # Fallback to basic text extraction
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            return soup.get_text(separator=' ').strip()
        except:
            return str(html_content)

def test_html_cleaning():
    """Test the HTML cleaning function with problematic examples."""
    
    test_cases = [
        # Original problem case
        "<h2>Bloedneus</h2><h2>Adele</h2><p>Kirsten Sokol:</p>",
        
        # Multiple headings with content
        "<h1>Titel</h1><h2>Subtitel</h2><p>Dit is de inhoud van het artikel.</p><p>Tweede paragraaf.</p>",
        
        # Mixed content with lists
        "<div><h3>Sectie</h3><ul><li>Item 1</li><li>Item 2</li></ul><p>Tekst na lijst.</p></div>",
        
        # Complex nested structure
        "<article><header><h1>Hoofdtitel</h1></header><section><h2>Sectie 1</h2><p>Tekst hier.</p></section></article>"
    ]
    
    print("🧪 Testing HTML cleaning with improved spacing:")
    print("=" * 60)
    
    for i, test_html in enumerate(test_cases, 1):
        print(f"\n🔍 Test case {i}:")
        print(f"   Input:  {test_html}")
        
        cleaned = clean_html_content_improved(test_html)
        print(f"   Output: '{cleaned}'")
        
        # Check for word concatenation issues
        words = cleaned.split()
        if len(words) > 0:
            print(f"   Words:  {words}")
            
            # Flag potential concatenation (words longer than typical)
            long_words = [w for w in words if len(w) > 15]
            if long_words:
                print(f"   ⚠️ Long words (potential concatenation): {long_words}")
        print("-" * 60)

# Run the test
test_html_cleaning()

🧪 Testing HTML cleaning with improved spacing:

🔍 Test case 1:
   Input:  <h2>Bloedneus</h2><h2>Adele</h2><p>Kirsten Sokol:</p>
   Output: 'Bloedneus Adele Kirsten Sokol:'
   Words:  ['Bloedneus', 'Adele', 'Kirsten', 'Sokol:']
------------------------------------------------------------

🔍 Test case 2:
   Input:  <h1>Titel</h1><h2>Subtitel</h2><p>Dit is de inhoud van het artikel.</p><p>Tweede paragraaf.</p>
   Output: 'Titel Subtitel Dit is de inhoud van het artikel. Tweede paragraaf.'
   Words:  ['Titel', 'Subtitel', 'Dit', 'is', 'de', 'inhoud', 'van', 'het', 'artikel.', 'Tweede', 'paragraaf.']
------------------------------------------------------------

🔍 Test case 3:
   Input:  <div><h3>Sectie</h3><ul><li>Item 1</li><li>Item 2</li></ul><p>Tekst na lijst.</p></div>
   Output: 'Sectie Item 1 Item 2 Tekst na lijst.'
   Words:  ['Sectie', 'Item', '1', 'Item', '2', 'Tekst', 'na', 'lijst.']
------------------------------------------------------------

🔍 Test case 4:
   Input:  <article><

## Text Preprocessing and Validation

Implement text cleaning and validation functions to remove URLs, email addresses, and normalize text while preserving readability.

In [4]:
def preprocess_text_for_export(text):
    """
    Clean and preprocess text for export while maintaining readability.
    
    Args:
        text (str): Clean text from HTML cleaning
        
    Returns:
        str: Preprocessed text ready for storage
    """
    if not text:
        return ""
    
    # Remove URLs but keep the text readable
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    
    # Remove email addresses but keep structure
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
    
    # Clean up excessive whitespace but preserve paragraphs
    text = re.sub(r'[ \t]+', ' ', text)  # Multiple spaces/tabs to single space
    text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)  # Multiple newlines to double newline
    
    # Remove leading/trailing whitespace from lines
    lines = [line.strip() for line in text.splitlines()]
    text = '\n'.join(line for line in lines if line)  # Remove empty lines
    
    return text.strip()


def validate_text_content(text, min_length=20, max_length=50000):
    """
    Validate text content for storage.
    
    Args:
        text (str): Text to validate
        min_length (int): Minimum text length
        max_length (int): Maximum text length
        
    Returns:
        tuple: (is_valid, reason)
    """
    if not text or not text.strip():
        return False, "Empty or whitespace-only text"
    
    text = text.strip()
    
    if len(text) < min_length:
        return False, f"Text too short ({len(text)} < {min_length} characters)"
    
    if len(text) > max_length:
        return False, f"Text too long ({len(text)} > {max_length} characters)"
    
    # Check for reasonable text content (not just repeated characters)
    unique_chars = len(set(text.lower()))
    if unique_chars < 10:
        return False, f"Text lacks diversity ({unique_chars} unique characters)"
    
    return True, "Valid text content"

# Test the preprocessing functions
print("🧪 Testing text preprocessing functions:")

test_text = """Dit is een test artikel met http://example.com en contact@example.nl.

Hier is een nieuwe paragraaf     met veel spaties.


En hier is nog een paragraaf na veel lege regels."""

print(f"Original text:\n{repr(test_text)}")

processed = preprocess_text_for_export(test_text)
print(f"\nProcessed text:\n{repr(processed)}")

is_valid, reason = validate_text_content(processed)
print(f"\nValidation: {is_valid} - {reason}")

🧪 Testing text preprocessing functions:
Original text:
'Dit is een test artikel met http://example.com en contact@example.nl.\n\nHier is een nieuwe paragraaf     met veel spaties.\n\n\nEn hier is nog een paragraaf na veel lege regels.'

Processed text:
'Dit is een test artikel met [URL] en [EMAIL].\nHier is een nieuwe paragraaf met veel spaties.\nEn hier is nog een paragraaf na veel lege regels.'

Validation: True - Valid text content


## SQLite Storage (Native Schema)

Store each article's cleaned text in a single SQLite database using explicit columns. The database has one row per article with columns:

- article_id (dataset index)
- title
- published_time (ISO8601), published_timestamp (epoch seconds)
- content (cleaned text), text_length
- processing_extracted_date, extraction_method


In [5]:
def _to_py(value):
    """Convert numpy/pandas scalars to native Python types for SQLite."""
    if isinstance(value, (np.integer, )):
        return int(value)
    if isinstance(value, (np.floating, )):
        return float(value)
    if isinstance(value, (pd.Timestamp, datetime)):
        return value.isoformat()
    return None if pd.isna(value) else value


def init_database(db_path):
    """
    Initialize SQLite database and return a connection.
    Creates table and indexes if they do not exist.
    Applies performance PRAGMAs suitable for bulk inserts.
    """
    db_path = str(db_path)
    Path(db_path).parent.mkdir(parents=True, exist_ok=True)
    conn = sqlite3.connect(db_path)

    # Performance-related PRAGMAs
    conn.execute("PRAGMA journal_mode=WAL;")          # Better concurrency and write throughput
    conn.execute("PRAGMA synchronous=NORMAL;")         # Good balance of safety/speed for WAL
    conn.execute("PRAGMA temp_store=MEMORY;")
    conn.execute("PRAGMA cache_size = -100000;")       # ~100MB page cache (negative => KB)
    conn.execute("PRAGMA page_size = 4096;")           # Default page size; effective on new DBs
    conn.execute("PRAGMA mmap_size = 3000000000;")     # Use memory-mapped I/O when available (~3GB)
    conn.execute("PRAGMA busy_timeout = 5000;")        # Wait for locks for up to 5s
    # conn.execute("PRAGMA threads = 4;")              # Optional (depends on SQLite build)

    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS articles (
            article_id TEXT PRIMARY KEY,
            title TEXT,
            published_time TEXT,
            published_timestamp REAL,
            content TEXT,
            text_length INTEGER,
            processing_extracted_date TEXT,
            extraction_method TEXT
        )
        """
    )
    conn.execute("CREATE INDEX IF NOT EXISTS idx_articles_published_time ON articles(published_time);")
    conn.execute("CREATE INDEX IF NOT EXISTS idx_articles_published_ts ON articles(published_timestamp);")
    return conn


def _upsert_sql():
    return (
        """
        INSERT INTO articles (
            article_id, title, published_time, published_timestamp,
            content, text_length, processing_extracted_date, extraction_method
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ON CONFLICT(article_id) DO UPDATE SET
            title=excluded.title,
            published_time=excluded.published_time,
            published_timestamp=excluded.published_timestamp,
            content=excluded.content,
            text_length=excluded.text_length,
            processing_extracted_date=excluded.processing_extracted_date,
            extraction_method=excluded.extraction_method
        """
    )


def _prepare_values(row, clean_text, processing_date=None):
    if processing_date is None:
        processing_date = datetime.now().isoformat()
    title = row.get('title', 'untitled')
    published_time = row.get('published_time')
    if pd.notna(published_time):
        if isinstance(published_time, str):
            pub_dt = pd.to_datetime(published_time)
        else:
            pub_dt = published_time
        published_time_iso = pub_dt.isoformat()
        published_ts = pub_dt.timestamp()
    else:
        published_time_iso = None
        published_ts = None

    article_id = str(row.name)
    return (
        article_id,
        _to_py(title) if title is not None else None,
        published_time_iso,
        published_ts,
        clean_text,
        int(len(clean_text)),
        processing_date,
        'improved_html_cleaning',
    )


def save_article_to_db(row, clean_text, conn):
    """
    Insert or update a single article row into the SQLite database (native columns).
    Returns: (success: bool, article_id: str, message: str)
    """
    try:
        values = _prepare_values(row, clean_text)
        with closing(conn.cursor()) as cur:
            cur.execute(_upsert_sql(), values)
        return True, str(row.name), "Success"
    except Exception as e:
        return False, "", str(e)


def process_single_article(row, conn, verbose=False):
    """
    Process a single article and write into SQLite DB (native schema).
    
    Args:
        row: DataFrame row with article data
        conn: SQLite connection
        verbose (bool): Whether to print detailed output
        
    Returns:
        dict: Processing results
    """
    result = {
        'success': False,
        'article_id': '',
        'text_length': 0,
        'error': None
    }
    
    try:
        # Clean HTML content only (title and description are already in the HTML)
        final_text = ""
        if pd.notna(row.get('content')):
            clean_content = clean_html_content_improved(row['content'])
            if clean_content:
                final_text = preprocess_text_for_export(clean_content)
        
        # Validate text
        is_valid, validation_reason = validate_text_content(final_text)
        if not is_valid:
            result['error'] = f"Text validation failed: {validation_reason}"
            return result
        
        # Save to DB
        success, article_id, message = save_article_to_db(row, final_text, conn)
        
        if success:
            result['success'] = True
            result['article_id'] = article_id
            result['text_length'] = len(final_text)
            if verbose:
                print(f"✅ Saved: {article_id} ({len(final_text)} chars)")
        else:
            result['error'] = f"DB save failed: {message}"
            if verbose:
                print(f"❌ Failed to save: {message}")
        
    except Exception as e:
        result['error'] = str(e)
        if verbose:
            print(f"❌ Error processing article: {e}")
    
    return result

# Test the DB functions on a single row
print("🧪 Testing SQLite write for a single article:")

if 'df' in locals() and df is not None and len(df) > 0:
    test_row = df.iloc[0]
    test_db_path = Path("output") / "test_articles.sqlite"
    
    conn = init_database(test_db_path)
    try:
        print(f"Testing with article: {test_row.get('title', 'No title')}")
        print(f"Publication date: {test_row.get('published_time', 'No date')}")
        
        result = process_single_article(test_row, conn, verbose=True)
        
        print(f"\nTest Result:")
        print(f"  Success: {result['success']}")
        print(f"  Article ID: {result['article_id']}")
        print(f"  Text length: {result['text_length']}")
        if result['error']:
            print(f"  Error: {result['error']}")
        
        # Verify row exists in DB
        if result['success']:
            with closing(conn.cursor()) as cur:
                cur.execute("SELECT COUNT(1) FROM articles WHERE article_id = ?", (result['article_id'],))
                count = cur.fetchone()[0]
                print(f"  Row in DB: {'✅' if count == 1 else '❌'} (count={count})")
    finally:
        conn.commit()
        conn.execute("PRAGMA optimize;")
        conn.close()
else:
    print("⚠️ No dataset loaded - cannot test DB write")

🧪 Testing SQLite write for a single article:
Testing with article: Euro nu ook in Litouwen
Publication date: 2015-01-01 00:32:52
✅ Saved: 1948 (1577 chars)

Test Result:
  Success: True
  Article ID: 1948
  Text length: 1577
  Row in DB: ✅ (count=1)


## Batch Processing Pipeline (SQLite)

Implement efficient batch processing to handle large datasets while writing directly to a SQLite database with transaction-per-batch for performance.

In [6]:
def process_articles_batch_sqlite(df, db_path, batch_size=1000, verbose=True):
    """
    Process articles in batches and save as rows in a SQLite database (native schema).
    One transaction per batch for performance.
    
    Returns stats dict.
    """
    print(f"🚀 Starting batch processing of {len(df):,} articles -> {db_path}")
    print(f"📊 Configuration:")
    print(f"   Batch size: {batch_size}")
    
    conn = init_database(db_path)
    stats = {
        'total_articles': len(df),
        'processed': 0,
        'successful': 0,
        'failed': 0,
        'skipped': 0,
        'total_text_length': 0,
        'processing_time': 0,
        'errors': []
    }
    
    start_time = time.time()

    # Precompile SQL and use a buffer for executemany
    upsert_sql = _upsert_sql()
    values_buffer = []
    buffer_target = 1000  # tuneable: number of rows per executemany
    
    try:
        for batch_start in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
            batch_end = min(batch_start + batch_size, len(df))
            batch_df = df.iloc[batch_start:batch_end]
            batch_success = 0
            batch_failed = 0
            
            with conn:  # implicit transaction
                cur = conn.cursor()
                try:
                    for idx, row in batch_df.iterrows():
                        try:
                            # Clean & validate
                            final_text = ""
                            if pd.notna(row.get('content')):
                                clean_content = clean_html_content_improved(row['content'])
                                if clean_content:
                                    final_text = preprocess_text_for_export(clean_content)
                            is_valid, validation_reason = validate_text_content(final_text)
                            stats['processed'] += 1
                            if not is_valid:
                                stats['failed'] += 1
                                batch_failed += 1
                                stats['errors'].append(f"Article {idx}: {validation_reason}")
                                continue
                            # Prepare values and buffer
                            values = _prepare_values(row, final_text)
                            values_buffer.append(values)
                            # Flush buffer when large enough
                            if len(values_buffer) >= buffer_target:
                                cur.executemany(upsert_sql, values_buffer)
                                values_buffer.clear()
                            stats['successful'] += 1
                            stats['total_text_length'] += len(final_text)
                            batch_success += 1
                        except Exception as e:
                            stats['failed'] += 1
                            batch_failed += 1
                            stats['errors'].append(f"Article {idx}: {str(e)}")
                    # Flush remaining in buffer at the end of the batch
                    if values_buffer:
                        cur.executemany(upsert_sql, values_buffer)
                        values_buffer.clear()
                finally:
                    cur.close()
            
            if verbose and (batch_start // batch_size) % 10 == 0:
                elapsed = time.time() - start_time
                rate = stats['processed'] / elapsed if elapsed > 0 else 0
                print(f"📊 Batch {batch_start//batch_size + 1}: {batch_success} successful, {batch_failed} failed")
                print(f"   Total progress: {stats['processed']:,}/{stats['total_articles']:,} ({rate:.1f} articles/sec)")

            # Periodically run optimize to update stats
            if (batch_start // batch_size) % 50 == 0:
                try:
                    conn.execute("PRAGMA optimize;")
                except Exception:
                    pass
    finally:
        try:
            conn.execute("PRAGMA optimize;")
        except Exception:
            pass
        conn.close()
    
    stats['processing_time'] = time.time() - start_time
    
    print(f"\n✅ Batch processing to SQLite completed!")
    print(f"📊 Final Statistics:")
    print(f"   Total articles: {stats['total_articles']:,}")
    print(f"   Successfully processed: {stats['successful']:,}")
    print(f"   Failed: {stats['failed']:,}")
    print(f"   Success rate: {(stats['successful']/max(1, stats['total_articles'])*100):.1f}%")
    print(f"   Total text extracted: {stats['total_text_length']:,} characters")
    print(f"   Processing time: {stats['processing_time']:.1f} seconds")
    print(f"   Average rate: {stats['processed']/max(1, stats['processing_time']):.1f} articles/second")
    
    if stats['errors']:
        print(f"\n⚠️ Error Summary (first 10):")
        for error in stats['errors'][:10]:
            print(f"   {error}")
        if len(stats['errors']) > 10:
            print(f"   ... and {len(stats['errors']) - 10} more errors")
    
    return stats

# Test batch processing with a small sample into SQLite
print("🧪 Testing batch processing pipeline (SQLite):")

if 'df' in locals() and df is not None:
    test_sample = df.head(10)
    test_db = Path("output") / "test_batch_articles.sqlite"
    print(f"Testing batch processing with {len(test_sample)} articles -> {test_db}...")
    test_stats = process_articles_batch_sqlite(
        df=test_sample,
        db_path=test_db,
        batch_size=5,
        verbose=True
    )
else:
    print("⚠️ No dataset loaded - cannot test batch processing")

🧪 Testing batch processing pipeline (SQLite):
Testing batch processing with 10 articles -> output\test_batch_articles.sqlite...
🚀 Starting batch processing of 10 articles -> output\test_batch_articles.sqlite
📊 Configuration:
   Batch size: 5


Processing batches: 100%|██████████| 2/2 [00:00<00:00, 181.85it/s]

📊 Batch 1: 5 successful, 0 failed
   Total progress: 5/10 (263.2 articles/sec)

✅ Batch processing to SQLite completed!
📊 Final Statistics:
   Total articles: 10
   Successfully processed: 10
   Failed: 0
   Success rate: 100.0%
   Total text extracted: 13,979 characters
   Processing time: 0.0 seconds
   Average rate: 10.0 articles/second


## Export All Articles to SQLite

Execute the complete pipeline to process all articles from the dataset and store them as rows in a SQLite database with clean text and core fields.

⚠️ WARNING: This may process 295k+ articles and take considerable time and disk space. Only run when ready!

In [7]:
# Full dataset processing -> SQLite - EXECUTE WITH CAUTION
print("🚀 FULL DATASET TEXT EXTRACTION TO SQLITE")
print("=" * 50)

if 'df' not in locals() or df is None:
    print("❌ Dataset not loaded. Please run the dataset loading cell first.")
else:
    print(f"📊 Dataset ready: {len(df):,} articles")
    print(f"📅 Date range: {df['published_time'].min()} to {df['published_time'].max()}")
    
    # Configuration
    FULL_DB_PATH = Path("output") / "articles_text_export.sqlite"
    BATCH_SIZE = 1000  # Process 1000 articles per batch
    
    print(f"\n⚙️ Configuration:")
    print(f"   Database path: {FULL_DB_PATH}")
    print(f"   Batch size: {BATCH_SIZE}")
    
    # Estimate disk space
    avg_text_per_article = 1500  # Rough estimate in characters
    total_estimated_text = len(df) * avg_text_per_article
    overhead_factor = 1.05  # index + row overhead
    estimated_size_mb = (total_estimated_text * overhead_factor) / (1024 * 1024)
    
    print(f"\n💾 Estimated storage requirements:")
    print(f"   Estimated text size: {estimated_size_mb:.1f} MB")
    print(f"   Number of rows: {len(df):,}")
    
    # Confirmation flag
    CONFIRM_PROCESSING = True
    
    if CONFIRM_PROCESSING:
        print(f"\n🏁 Starting full dataset processing -> SQLite...")
        try:
            final_stats = process_articles_batch_sqlite(
                df=df,
                db_path=FULL_DB_PATH,
                batch_size=BATCH_SIZE,
                verbose=True
            )
            print(f"\n🎉 FULL PROCESSING COMPLETED SUCCESSFULLY!")
            print(f"🗄️ Database saved to: {FULL_DB_PATH}")
        except KeyboardInterrupt:
            print(f"\n⚠️ Processing interrupted by user")
            print(f"💾 Partial results available in: {FULL_DB_PATH}")
        except Exception as e:
            print(f"\n❌ Error during processing: {e}")
            print(f"💾 Check database for partial results: {FULL_DB_PATH}")
    else:
        print(f"\n⏸️ Processing skipped (CONFIRM_PROCESSING = False)")
        print(f"💡 Set CONFIRM_PROCESSING = True to run the full export")

print(f"\n✅ Text extraction notebook execution complete!")

🚀 FULL DATASET TEXT EXTRACTION TO SQLITE
📊 Dataset ready: 295,259 articles
📅 Date range: 2015-01-01 00:32:52 to 2025-03-31 23:45:01

⚙️ Configuration:
   Database path: output\articles_text_export.sqlite
   Batch size: 1000

💾 Estimated storage requirements:
   Estimated text size: 443.5 MB
   Number of rows: 295,259

🏁 Starting full dataset processing -> SQLite...
🚀 Starting batch processing of 295,259 articles -> output\articles_text_export.sqlite
📊 Configuration:
   Batch size: 1000


Processing batches:   0%|          | 1/296 [00:01<05:08,  1.04s/it]

📊 Batch 1: 1000 successful, 0 failed
   Total progress: 1,000/295,259 (956.7 articles/sec)


Processing batches:   4%|▎         | 11/296 [00:09<04:01,  1.18it/s]

📊 Batch 11: 1000 successful, 0 failed
   Total progress: 11,000/295,259 (1162.7 articles/sec)


...

📊 Batch 281: 997 successful, 3 failed
   Total progress: 281,000/295,259 (967.2 articles/sec)


Processing batches:  98%|█████████▊| 291/296 [05:03<00:06,  1.26s/it]

📊 Batch 291: 997 successful, 3 failed
   Total progress: 291,000/295,259 (959.1 articles/sec)


Processing batches: 100%|██████████| 296/296 [05:08<00:00,  1.04s/it]


✅ Batch processing to SQLite completed!
📊 Final Statistics:
   Total articles: 295,259
   Successfully processed: 295,097
   Failed: 162
   Success rate: 99.9%
   Total text extracted: 713,883,339 characters
   Processing time: 308.5 seconds
   Average rate: 957.2 articles/second

⚠️ Error Summary (first 10):
   Article 65832: Text too long (59172 > 50000 characters)
   Article 184098: Text too long (50351 > 50000 characters)
   Article 184168: Text too long (52950 > 50000 characters)
   Article 184244: Text too long (76183 > 50000 characters)
   Article 184308: Text too long (66483 > 50000 characters)
   Article 184346: Text too long (53228 > 50000 characters)
   Article 184840: Text too long (52559 > 50000 characters)
   Article 185219: Text too long (64243 > 50000 characters)
   Article 185334: Text too long (73335 > 50000 characters)
   Article 185456: Text too long (59028 > 50000 characters)
   ... and 152 more errors

🎉 FULL PROCESSING COMPLETED SUCCESSFULLY!
🗄️ Database saved t

## Summary and Next Steps

This notebook extracts clean text from the Dutch news articles dataset and stores it efficiently in a SQLite database with a simple, database-native schema.

### ✅ Key Features Implemented

1. Improved HTML cleaning with proper spacing
2. Clean text stored in a single SQLite file
3. Native schema with explicit columns
4. Fast querying using index on published_time and primary key
5. Batch processing with transaction-per-batch for performance

### 🗄️ SQLite Schema

Table: articles
- article_id TEXT PRIMARY KEY (DataFrame index)
- title TEXT
- published_time TEXT (ISO8601)
- published_timestamp REAL (epoch seconds)
- content TEXT (cleaned)
- text_length INTEGER
- processing_extracted_date TEXT (ISO8601)
- extraction_method TEXT

### 🔍 Example Queries

- Count rows: SELECT COUNT(*) FROM articles;
- Latest articles: SELECT article_id, title, published_time FROM articles ORDER BY published_timestamp DESC LIMIT 20;
- Year filter: SELECT COUNT(*) FROM articles WHERE published_time LIKE '2015%';
- Search word: SELECT article_id, title FROM articles WHERE content LIKE '%verkiezing%';

### 🚀 Optional

- Add full-text search with FTS5 for large-scale content search:
  - CREATE VIRTUAL TABLE articles_fts USING fts5(article_id, content, content='articles', content_rowid='rowid');
  - INSERT INTO articles_fts(article_id, content) SELECT article_id, content FROM articles;